In [35]:
import json
import os
import subprocess
import re
import csv

In [36]:
# ---- PATH CONFIGURATION ----
graph_type = "3Reg"
n_vertices = 100

#base_dir = os.path.expanduser("~/Projects/JuliaMPS/BiqCrunch")  # adjust if needed
biq_path = os.path.join("..", "BiqCrunch", "problems", "max-cut", "biqcrunch")
param_path = os.path.join("..", "BiqCrunch", "problems", "max-cut", "biq_crunch.param")
converter_path = os.path.join("..", "BiqCrunch", "tools", "mc2bc.py")

dir_path = f"../data/MaxCut/{graph_type}/{n_vertices}v/graphs"
save_path = f"../benchmark/BiqCrunch/results/MaxCut/{graph_type}/graphs"
os.makedirs(save_path, exist_ok=True)

# ---- CONVERT JSON → .mc ----
def json_to_mc(json_path, mc_path):
    with open(json_path, "r") as f:
        data = json.load(f)
    N = data["N"]
    edges = data["edges"]
    with open(mc_path, "w") as f:
        f.write(f"{N} {len(edges)}\n")
        for k, w in edges.items():
            i, j = map(int, k.split(","))
            f.write(f"{i+1} {j+1} {w}\n")

# ---- RUN BIQCRUNCH ----
def convert_mc_to_bc(mc_path):
    """Convert .mc → .bc"""
    bc_path = mc_path.replace(".mc", ".bc")

    # Convert to .bc format
    with open(bc_path, "w") as f:
        subprocess.run(["python3", converter_path, mc_path], stdout=f, check=True)

    return

def run_biqcrunch(bc_path):
    """Run BiqCrunch on .bc file and return the output."""
    result = subprocess.run([biq_path, bc_path, param_path], capture_output=True, text=True)
    return result.stdout

def parse_output(path):
    with open(path, "r") as f:
        text = f.read()

    # Try to match both numbers
    match_val = re.search(r"Maximum value\s*=\s*([-\d.]+)", text)
    match_bound = re.search(r"Root node bound\s*=\s*([-\d.]+)", text)

    value = float(match_val.group(1)) if match_val else None
    bound = float(match_bound.group(1)) if match_bound else None

    gap = None
    if value is not None and bound is not None:
        gap = 100 * (bound - value) / abs(value)

    return value, bound, gap

# ---- MAIN LOOP ----
results = []
max_idx = 20
for idx in range(max_idx):
    json_path = os.path.join(dir_path, f"graph{idx}.json")
    mc_path = os.path.join(save_path, f"graph{idx}.mc")

    if not os.path.exists(json_path):
        print(f"Missing: {json_path}")
        continue

    json_to_mc(json_path, mc_path)
    print(f"Running BiqCrunch on {mc_path}...")
    convert_mc_to_bc(mc_path)

    bc_path = mc_path.replace(".mc", ".bc")
    run_biqcrunch(bc_path)

    output_path = bc_path.replace(".bc", ".bc.output")
    value, bound, gap = parse_output(output_path)
    print(f"Value: {value}, Bound: {bound}, Gap: {gap}%")

Running BiqCrunch on ../benchmark/BiqCrunch/results/MaxCut/3Reg/graphs/graph0.mc...
Value: 137.0, Bound: 137.6992, Gap: 0.5103649635036427%
Running BiqCrunch on ../benchmark/BiqCrunch/results/MaxCut/3Reg/graphs/graph1.mc...
Value: 138.0, Bound: 138.96817, Gap: 0.7015724637681061%
Running BiqCrunch on ../benchmark/BiqCrunch/results/MaxCut/3Reg/graphs/graph2.mc...
Value: 136.0, Bound: 136.6694, Gap: 0.4922058823529382%
Running BiqCrunch on ../benchmark/BiqCrunch/results/MaxCut/3Reg/graphs/graph3.mc...
Value: 139.0, Bound: 139.99497, Gap: 0.7158057553956799%
Running BiqCrunch on ../benchmark/BiqCrunch/results/MaxCut/3Reg/graphs/graph4.mc...
Value: 137.0, Bound: 137.65999, Gap: 0.48174452554744046%
Running BiqCrunch on ../benchmark/BiqCrunch/results/MaxCut/3Reg/graphs/graph5.mc...
Value: 139.0, Bound: 139.66061, Gap: 0.4752589928057491%
Running BiqCrunch on ../benchmark/BiqCrunch/results/MaxCut/3Reg/graphs/graph6.mc...
Value: 136.0, Bound: 136.74868, Gap: 0.5505000000000054%
Running BiqCru

In [37]:
csv_path = os.path.join(save_path, "biqcrunch_results.csv")

rows = []
for idx in range(max_idx + 1):
    fname = f"graph{idx}.bc.output"
    fpath = os.path.join(save_path, fname)
    if not os.path.exists(fpath):
        print(f"Missing: {fpath}")
        continue

    value, bound, gap = parse_output(fpath)
    rows.append({
            "file": fname,
            "cost": value,
            "bound": bound,
            "gap_percent": gap
        })
    print(f"Parsed {fname}: cost={value}, bound={bound}, gap={gap:.3f}%")

# Write CSV
with open(csv_path, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["file", "cost", "bound", "gap_percent"])
    writer.writeheader()
    writer.writerows(rows)

print(f"\nSaved results to {csv_path}")

Parsed graph0.bc.output: cost=137.0, bound=137.6992, gap=0.510%
Parsed graph1.bc.output: cost=138.0, bound=138.96817, gap=0.702%
Parsed graph2.bc.output: cost=136.0, bound=136.6694, gap=0.492%
Parsed graph3.bc.output: cost=139.0, bound=139.99497, gap=0.716%
Parsed graph4.bc.output: cost=137.0, bound=137.65999, gap=0.482%
Parsed graph5.bc.output: cost=139.0, bound=139.66061, gap=0.475%
Parsed graph6.bc.output: cost=136.0, bound=136.74868, gap=0.551%
Parsed graph7.bc.output: cost=137.0, bound=137.85389, gap=0.623%
Parsed graph8.bc.output: cost=137.0, bound=137.81543, gap=0.595%
Parsed graph9.bc.output: cost=136.0, bound=136.64573, gap=0.475%
Parsed graph10.bc.output: cost=137.0, bound=137.58659, gap=0.428%
Parsed graph11.bc.output: cost=137.0, bound=137.53857, gap=0.393%
Parsed graph12.bc.output: cost=135.0, bound=135.4187, gap=0.310%
Parsed graph13.bc.output: cost=135.0, bound=135.82542, gap=0.611%
Parsed graph14.bc.output: cost=135.0, bound=135.94304, gap=0.699%
Parsed graph15.bc.outpu